In [111]:
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from scipy.signal import lfilter

def Information_Transmit():
    M = 2 # bits per symbol (i.e. 2 in QPSK modulation)
    Information_to_transmit = 'I worked all week on digital communications and all I got was this sequence of ones and zeros.'
    binary = ''.join(format(ord(i), '08b') for i in Information_to_transmit)
    data_bits = np.zeros((len(binary),))
    for i in range(len(binary)):
        data_bits[i] = binary[i]
    
    # Add synch_word to
    sync_word = np.asarray([1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0])
    print('sync_word:',sync_word)
    bit_sequence = np.hstack([sync_word, data_bits])
    
    # Add Preamble
    preamle_code = np.asarray([1,1,0,0])
    for i in range(16):
        if i ==0:
             preamble_swap = preamle_code
        else:    
            preamble = np.hstack([preamble_swap, preamle_code])
            preamble_swap = preamble
    
    print('preamble:',preamble)
    print('length of preamble:', len(preamble))
    QPSK_frame = np.hstack([preamble, bit_sequence])
    
    print('QPSK Frame:',len(QPSK_frame))
    # Convert serial data to parallel
    def Serial_to_Parallel(x):
        return x.reshape((len(x)//M, M))
    QPSK_bits = Serial_to_Parallel(QPSK_frame)

    return QPSK_bits

In [112]:
def lut(data, inputVec, outputVec):
    if len(inputVec) != len(outputVec):
        print('Input and Output vectors must have identical length')
    # Initialize output
    output = np.zeros(data.shape)
    # For each possible data value
    eps = np.finfo('float').eps
    for i in range(len(inputVec)):
        # Find the indices where data is equal to that input value
        for k in range(len(data)):
            if abs(data[k]-inputVec[i]) < eps:
                # Set those indices in the output to be the appropriate output value.
                output[k] = outputVec[i]
    return output

In [113]:
def oversample(x, OS_Rate):
    # Initialize output
    length = len(x[0])
    x_s = np.zeros((1,length*OS_Rate))
    # Fill in one out of every OS_Rate samples with the input values
    count = 0
    h = 0
    for k in range(len(x_s[0])):
        count = count + 1
        if count == OS_Rate:
            x_s[0][k] = x[0][h]
            count = 0
            h = h + 1
    return x_s

In [114]:
def SRRC(alpha, N, Lp):
    # Add epsilon to the n values to avoid numerical problems
    ntemp = list(range(-N*Lp, N*Lp+1))
    n = []
    for each in ntemp:
        n.append(each + math.pow(10,-9))
    # Plug into time domain formula for the SRRC pulse shape
    h = []
    coeff = 1/math.sqrt(N)
    for each in n:
        sine_term = math.sin(math.pi * each * (1-alpha) / N)
        cosine_term = math.cos(math.pi * each * (1+alpha) / N)
        cosine_coeff = 4 * alpha * each / N
        numerator = sine_term + (cosine_coeff * cosine_term)
        denom_coeff = math.pi * each / N
        denom_part = 1 - math.pow(cosine_coeff, 2)
        denominator = denom_coeff * denom_part
        pulse = coeff * numerator / denominator
        h.append(pulse)
    return h

In [115]:
Frame = Information_Transmit()
data1=[]
for i in range(len(Frame)):
    data1.append(2*Frame[i][0]+Frame[i][1])
data = np.array(data1)

sync_word: [1 1 1 0 1 0 1 1 1 0 0 1 0 0 0 0]
preamble: [1 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0 1
 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0]
length of preamble: 64
QPSK Frame: 832


In [116]:
###########################################
### Modulation
### INPUT: data
### OUTPUT: modulated values, x

A = math.sqrt(9/2)
inputVec   = [0, 1, 2, 3]
outputVecI = [A, -A, A, -A]
outputVecQ = [A, A, -A, -A]
xI         = lut(data, inputVec, outputVecI)
xQ         = lut(data, inputVec, outputVecQ)
xI = xI.reshape((1,len(data)))
xQ = xQ.reshape((1,len(data)))

In [117]:
###########################################
### Upsample
### INPUT: modulated values, x
### OUTPUT: modulated values at sampling rate, x_s

x_s_I = oversample(xI,8)
x_s_Q = oversample(xQ,8)

In [118]:
pulse = SRRC(0.5, 8, 6)
pulse = np.array(pulse)
pulse = np.reshape(pulse, pulse.size)
x_s_I = np.reshape(x_s_I, x_s_I.size)
x_s_Q = np.reshape(x_s_Q, x_s_Q.size)
s_0_I = np.convolve(x_s_I, pulse, mode='full')
s_0_Q = np.convolve(x_s_Q, pulse, mode='full')

In [119]:
QPSK_samples = s_0_I + s_0_Q*1j

QPSK_samples: 3424
type: <class 'numpy.complex128'>


In [121]:
## Add 1024 zero value samples at the beginning
QPSK_samples_Final = np.hstack([np.zeros(1024, dtype=np.complex128),QPSK_samples])

4448


In [122]:
def write_complex_binary(data, filename):
    '''
    Open filename and write array to it as binary
    Format is interleaved float IQ e.g. each I,Q should be 32-bit float 
    INPUT
    ----
    data:     data to be wrote into the file. format: (length, )
    filename: file name
    '''
    re = np.real(data)
    im = np.imag(data)
    binary = np.zeros(len(data)*2, dtype=np.float32)
    binary[::2] = re
    binary[1::2] = im
    binary.tofile(filename)   
write_complex_binary(QPSK_samples_Final, 'QPSK_signal.iq')

In [123]:
def get_samps_from_file(filename): 
    '''
    load samples from the binary file
    '''
    # File should be in GNURadio's format, i.e., interleaved I/Q samples as float32
    samples = np.fromfile(filename, dtype=np.float32)
    samps = (samples[::2] + 1j*samples[1::2]).astype((np.complex64)) # convert to IQIQIQ  
    return samps

In [124]:
get_samps_from_file('QPSK_signal.iq')

array([0.        +0.j        , 0.        +0.j        ,
       0.        +0.j        , ..., 0.00127108-0.00127108j,
       0.00283863-0.00283863j, 0.00333891-0.00333891j], dtype=complex64)